In [1]:
#ΤΟ ΘΕΛΩ ΠΑΝΤΟΤΕ
! pip install -U scikit-learn
#ΤΟ ΘΕΛΩ ΓΙΑ ΝΑ ΚΑΝΩ IMBALANCE ΤΟ DATASET ΜΟΥ
! pip install -U imbalanced-learn

Requirement already up-to-date: scikit-learn in c:\users\user\anaconda3\lib\site-packages (0.22.2.post1)


Requirement already up-to-date: imbalanced-learn in c:\users\user\anaconda3\lib\site-packages (0.6.2)


In [2]:
#ΧΡΗΣΙΜΑ ΕΡΓΑΛΕΙΑ ΠΟΥ ΑΡΓΑ Η ΓΡΗΓΟΡΑ ΘΑ ΤΑ ΤΑ ΧΡΕΙΑΣΤΕΙΣ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#ΧΡΗΣΙΜΟ ΓΙΑ ΝΑ ΠΑΡΕΙΣ ΓΡΑΦΙΚΕΣ ΠΑΡΑΣΤΑΣΕΙΣ
#import seaborn as sns

#ΕΡΓΑΛΕΙΑ ΓΙΑ ΝΑ ΚΑΝΕΙς IMPUTE TO DATASET ΣΟΥ ΔΗΛΑΔΗ ΝΑ ΓΕΜΊΣΕΙΣ ΜΕ ΤΙΜΕΣ ΤΑ NaN ΚΕΛΙΑ - ΛΕΙΤΟΥΡΓΕΙ ΓΙΑ ΑΡΙΘΜΗΤΙΚΕς ΤΙΜΕΣ
#from sklearn.impute import SimpleImputer 
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer

#ΓΙΑ ΝΑ ΚΑΝΕΙΣ LABEL ENCODER
from sklearn.preprocessing import LabelEncoder

#ΟΤΑΝ ΘΕΛΕΙΣ ΝΑ ΚΑΝΕΙΣ SPLIT ΤΟ DATASET ΣΟΥ ΣΕ TRAIN-TEST
from sklearn.model_selection import train_test_split

#ΓΙΑ ΜΕΙΩΣΗ ΔΙΑΣΤΑΤΙΚΟΤΗΤΑΣ
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

#ΕΡΓΑΛΕΙΑ ΓΙΑ ΝΑ ΠΡΑΓΜΑΤΟΠΟΙΗΣΩ ΙΜΒΑLANCE - OVERSAMPLING - UNDERSAMPLING
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE #ΠΡΟΚΑΛΕΙ ΘΕΜΑΤΑ ΟΤΑΝ ΕΧΟΥΜΕ ΤΕΡΑΣΤΙΕΣ ΔΙΑΦΟΡΕΣ , ΧΡΗΣΙΜΟΠΟΙΕΙ KNN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import ClusterCentroids

In [3]:
from sklearn.datasets import fetch_kddcup99
fetch=fetch_kddcup99()
fetch_data=fetch.data
fetch_target=fetch.target
fetch_Descr=fetch.DESCR
fetch_data=pd.DataFrame(data=fetch_data)
fetch_target=pd.DataFrame(data=fetch_target,columns=['target'])
fetch_data=pd.concat([fetch_data, fetch_target], axis=1)
#ONE HOT ENCODING ΓΙΑ ΤΙΣ ΣΤΗΛΕΣ ΠΟΥ ΒΛΕΠΟΥΜΕ ΟΤΙ ΜΑΣ ΕΝΟΧΛΟΥΝ
fetch_data=pd.get_dummies(fetch_data,columns=[1,2,3])

#ΤΟΠΟΘΕΤΟΥΜΕ ΤΗΝ ΣΤΗΛΗ Τarget ΣΤΟ ΤΕΛΟΣ ΓΙΑΤΙ ΜΑΣ ΕΞΥΠΗΡΕΤΕΙ
cols = list(fetch_data.columns.values) 
cols.pop(cols.index('target')) 
fetch_data = fetch_data[cols+['target']]

#LABEL ENCODING ΓΙΑ ΤΟ Target
labelencoder = LabelEncoder()
fetch_data.target= labelencoder.fit_transform(fetch_data.target)
fetch_data.head()

,0,4,5,6,7,8,9,10,11,12,...,3_b'RSTO',3_b'RSTOS0',3_b'RSTR',3_b'S0',3_b'S1',3_b'S2',3_b'S3',3_b'SF',3_b'SH',target
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,11
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,11
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,11
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,11
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,11


## Εχοντας ολοκληρώσει την BeforeSplit διαδικασία πλεον χωριζουμε το DataSet μας σε train - test και εφαρμόζουμε ορισμένες τεχνικές για βελτιστοποίηση

In [4]:
fetch_data.shape

(494021, 119)

In [5]:
fetch_features_df=fetch_data.iloc[:,:118]
fetch_labels_df=fetch_data.iloc[:,[118]]

In [6]:
fetch_train, fetch_test, fetch_train_labels, fetch_test_labels = train_test_split(fetch_features_df, fetch_labels_df, test_size=0.2,random_state=1)

## Standardization

Η κανονικοποίηση των χαρακτηριστικών μπορεί να γίνει με 2 βασικούς τρόπους, γνωστούς και από τη στατιστική. Με την διαίρεση με τη διαφορά μεγίστου-ελαχίστου (feature scaling) οπότε οι τιμές όλων των χαρακτηριστικών κλιμακώνονται γραμμικά στο διάστημα [0,1] ή με το z-score (ή standard score) του κάθε χαρακτηριστικού (standardization), που κάνει το χαρακτηριστικό να έχει μέση τιμή μηδέν και διακύμανση μονάδα, σαν την κανονική κατανομή. 

Η μετατροπή μεγίστου ελαχίστου γίνεται με τον τύπο $$X' = {{X - X_{min}} \over {X_{max} - X_{min}}}$$Η μετατροπή σε standard score γίνεται με τον τύπο: $$z = {{X- \mu }\over \sigma}$$ όπου: $μ$ είναι η μέση τιμή του χαρακτηριστικού και $σ$ η απόκλιση. Στην πράξη δεν μας ενδιαφέρει αν η πραγματική κατανομή των χαρακτηριστικών είναι κανονική, απλά αφαιρούμε τη μέση τιμή και διαιρούμε με την απόκλιση για να έχουν τα χαρακτηριστικά της.

H μετατροπή σε standard score είναι απαραίτητη σε πολλούς ταξινομητές για να συμπεριφερθούν σωστά. Επίσης είναι πιο ανθεκτική από την min max σε τιμές outliers δηλαδή σποραδικές τιμές που είναι πολύ μακριά απο τη μέση τιμή και τις υπόλοιπες τιμές του χαρακτηριστικού (η min max θα συμπιέσει τις περισσότερες τιμές σε ένα μικρό διάστημα)

Από την άλλη η κλιμάκωση σε [0,1] είναι λιγότερο ευαίσθητη σε πολυ μικρές αποκλίσεις και επίσης σε αραιά (sparse) διανύσματα χαρακτηριστικών (δηλαδή με πολλές μηδενικες τιμές) η εφαρμογή της διατηρεί τα μηδέν, κάτι που μπορεί να είναι καθοριστικό για την ταχύτητα εκπαίδευσης.

In [7]:
#SCALED STANDARD ΓΙΑ ΟΛΕΣ ΤΙΣ ΠΑΡΑΜΕΤΡΟΥΣ : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
def scaled_Standard(train,test):
    cols1=train.columns
    cols2=test.columns
    scaler = preprocessing.StandardScaler().fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    train_scaled=pd.DataFrame(data=train_scaled,columns=cols1)
    test_scaled=pd.DataFrame(data=test_scaled,columns=cols2)
    return train_scaled,test_scaled

#Min Max ΓΙΑ ΟΛΕΣ ΤΙΣ ΠΑΡΑΜΕΤΡΟΥΣ : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
def min_max(train,test):
    cols1=train.columns
    cols2=test.columns
    min_max_scaler = preprocessing.MinMaxScaler()
    train_scaled = min_max_scaler.fit_transform(train)
    test_scaled = min_max_scaler.transform(test)
    train_scaled=pd.DataFrame(data=train_scaled,columns=cols1)
    test_scaled=pd.DataFrame(data=test_scaled,columns=cols2)
    return train_scaled,test_scaled

## Variance Threshold

!! προσοχή όταν χρησιμοποιηθεί με one hot encoder [0,1] μπορεί να σβήσει αρκετά χαρακτηριστικά αν το threshold ξεπεράσει ένα όριο !!

In [8]:
# ΚΟΒΕΙ ΟΣΑ ΧΑΡΑΚΤΗΡΙΣΤΙΚΑ ΕΧΟΥΝ ΔΙΑΚΥΜΑΝΣΗ ΠΑΝΩ ΑΠΟ ΤΟ ΔΟΘΕΝ ΟΡΙΟ 
def variance(train,test,thresh):
    cols1=train.columns
    cols2=test.columns
    variance_thresh = VarianceThreshold(thresh)
    x_train = variance_thresh.fit_transform(train)
    x_test = variance_thresh.transform(test)
    x_train=pd.DataFrame(data=x_train,columns=cols1)
    x_test=pd.DataFrame(data=x_test,columns=cols2)
    return x_train, x_test

## IMBALANCE YOUR DATA :

 * Πρώτα βλέπεις πόσες μεταβλητές έχεις --> 23

In [9]:
fetch_train_labels.nunique()

target    23
dtype: int64

* Μετά βλέπεις τις τιμές την συχνότητα εμφάνισης τους 

In [10]:
print('Max variable=',fetch_train_labels['target'].value_counts().idxmax(),'with frequency=',fetch_train_labels['target'].value_counts().max())
print('Min variable=',fetch_train_labels['target'].value_counts().idxmin(),'with frequency=',fetch_train_labels['target'].value_counts().min())
print('ratio=',fetch_train_labels['target'].value_counts().min()*100/fetch_train_labels['target'].value_counts().max())

Max variable= 18 with frequency= 224652
Min variable= 12 with frequency= 2
ratio= 0.0008902658333778467


In [11]:
fetch_train_labels.apply(pd.value_counts)

,target
18,224652
9,85703
11,77850
0,1759
17,1282
5,989
15,853
21,820
20,775
14,204


* Αν έχεις ένα ratio <40% καλό θα ήταν να εξισορροπήσεις το dataset σου , το OverSample ανεβάζει τη συχνότητα των μικρών στα πλαίσια των μεγαλύτερων , το UnderSample ρίχνει τη συχνότητα των μεγάλων στα πλάισια των μικρών

In [12]:
#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html
#SMOTE synthesises new minority instances between existing (real) minority instances
#Finding the k-nearest-neighbors for minority class observations (finding similar observations)
def smoteoversampler(train,train_labels):
    cols1=train.columns
    cols2=train_labels.columns
    sm = SMOTE()
    x_train_res, y_train_res = sm.fit_sample(train, train_labels)
    x_train_res=pd.DataFrame(data=x_train_res,columns=cols1)
    y_train_res=pd.DataFrame(data=y_train_res,columns=cols2)
    return x_train_res, y_train_res

#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.RandomOverSampler.html
#ΠΡΟΣΕΧΕ ΓΙΑ OVERFITTING
#Randomly duplicate examples in the minority class.
def randomoversampler(train,train_labels):
    cols1=train.columns
    cols2=train_labels.columns
    ros = RandomOverSampler()
    X_ros, y_ros = ros.fit_sample(train, train_labels)
    X_ros=pd.DataFrame(data=X_ros,columns=cols1)
    y_ros=pd.DataFrame(data=y_ros,columns=cols2)
    return X_ros, y_ros

#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html
#ΠΡΟΣΕΧΕ ΜΗΝ ΧΑΣΕΙΣ ΧΡΗΣΙΜΕΣ ΠΛΗΡΟΦΟΡΙΕΣ
#Random undersampling deletes examples from the majority class
def randomundersampler(train,train_labels):
    cols1=train.columns
    cols2=train_labels.columns
    rus = RandomUnderSampler(return_indices=True)
    X_rus, y_rus, id_rus = rus.fit_sample(train,train_labels)
    X_rus=pd.DataFrame(data=X_rus,columns=cols1)
    y_rus=pd.DataFrame(data=y_rus,columns=cols2)
    return X_rus, y_rus

#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.TomekLinks.html
#Works with Links
#The procedure can be used to find all of those examples in the majority class that are closest to the minority class, then removed.
def tomekundersampler(train,train_labels):
    cols1=train.columns
    cols2=train_labels.columns
    tl = TomekLinks(return_indices=True, ratio='majority')
    X_tl, y_tl, id_tl = tl.fit_sample(train, train_labels)
    X_tl=pd.DataFrame(data=X_tl,columns=cols1)
    y_t1=pd.DataFrame(data=y_tl,columns=cols2)
    return X_tl, y_tl

#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.combine.SMOTETomek.html
#Resamples the dataset
def smotomek(train,train_labels):
    cols1=train.columns
    cols2=train_labels.columns
    smt = SMOTETomek(ratio='auto')
    X_smt, y_smt = smt.fit_sample(train, train_labels)
    X_smt=pd.DataFrame(data=X_smt,columns=cols1)
    y_smt=pd.DataFrame(data=y_smt,columns=cols2)
    return X_smt, y_smt

#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.ClusterCentroids.html
#Uses KMeans algorithm 
#UnderSamples by replacing a cluster of majority samples by the cluster centroid of a KMeans algorithm.
def clusterunder(train,train_labels):
    cols1=train.columns
    cols2=train_labels.columns
    cc = ClusterCentroids()
    X_cc, y_cc = cc.fit_sample(train, train_labels)
    X_cc=pd.DataFrame(data=X_cc,columns=cols1)
    y_cc=pd.DataFrame(data=y_cc,columns=cols2)
    return X_cc, y_cc

## Μείωση της διαστατικότητας , πέφτουν οι διαστάσεις του σετ μας με αποτέλεσμα να έχουμε καλύτερο αποτέλεσμα
!! Πολυ χρήσιμο όταν χρησιμοποιούμε one hot encoder !!

In [13]:
#n=25 , STO EPOMENO THA DOUME PWS VRISKOYME TH VELTISTH PARAMETRO
def pca(n,train,test):
    pcaa = PCA(n_components=n)
    trainPCA =  pcaa.fit_transform(train)
    testPCA = pcaa.transform(test)
    return trainPCA , testPCA